<a href="https://colab.research.google.com/github/dsk-003/Air-Quality/blob/main/final_23_jan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy-transformers

In [ ]:
!pip install -U spacy

In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [4]:
spacy.__version__

'3.7.2'

In [5]:
!nvidia-smi


Tue Jan 23 09:46:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8              10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
import json
cv_data=json.load(open('/content/drive/MyDrive/23-01-24_model/manual.json','r'))

In [7]:
len(cv_data)

19

In [8]:
cv_data[1]

['Sumit Singh EmailId sumit121singhgmailcom Contact 918375829395 skype Id  sumitsingh555 Skills  4 years experience product development  Experience Computer Vision NLP Machine Learning Deep Learning  Data Preparation Training model Model Evaluation Model Deployment  Image processing techniques using OpenCV  Experience Deep learning using Convolutional Neural Networks  CNN Recurrent Neural Network RNN BERT  Programming Language Python3  Frameworks Tensorflow keras PyTorch  Libraries OpenCV NLTK Sklearn  Experience perform operation GPU CPU  Algorithms CNN SVM Logistic Regression  Image Analysis Pattern Recognition  Show passion technologies cando attitude Experience Software Engineer Aug 2020 – March 2021 QuyTech Title – AI  ML Based Intelligent Video Advertisements Analysis Description implement tracking algorithms humans cars track car Human frame assign unique ID individual deep sort algorithm keep data memory old object detected frame algorithm assign old ID outcomes generated algor

In [9]:
def get_spacy_doc(file,data):
  nlp = spacy.blank('en')
  db = DocBin()

  for text,annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity==True:
        continue

      entity_indices = entity_indices + list(range(start, end))

      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')

      except:
        continue

      if span is None:
        err_data = str([start, end]) + "     " + str(text)  + "\n"
        file.write(err_data)

      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db

In [10]:
from sklearn.model_selection import train_test_split
import spacy
train,test = train_test_split(cv_data, test_size=0.2)

In [11]:
len(train),len(test)

(15, 4)

In [12]:
!python -m spacy init fill-config /content/drive/MyDrive/23-01-24_model/base_config.cfg config.cfg

2024-01-23 09:49:29.892811: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 09:49:29.892870: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 09:49:29.894115: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 09:49:30.914290: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [13]:
file = open('error.txt','w')

db = get_spacy_doc(file,train)
db.to_disk('train_data.spacy')


db = get_spacy_doc(file,test)
db.to_disk('test_data.spacy')

file.close()

100%|██████████| 4/4 [00:00<00:00, 34.80it/s]


In [14]:
len(db.tokens)

4

In [15]:
!python -m spacy train /content/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

2024-01-23 09:51:40.114609: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 09:51:40.114725: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 09:51:40.116300: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 09:51:41.671507: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
config.json: 100% 481/481 [00:00<00:00, 2.39MB/s]
vocab.json: 100% 899k/899

### Model test

In [16]:
nlp = spacy.load('/content/output/model-best')

In [17]:
doc = nlp('my name is Ritika Jadhav I worked at IBM . my skillset is expert in python programming machine learning deep learning data analysis NLP generative ai expert in scikit learn library.')

for ent in doc.ents:
  print(ent.text, "->>>>",ent.label_)

Ritika Jadhav ->>>> NAME
python programming machine learning deep learning data analysis NLP generative ai expert in scikit learn library. ->>>> SKILLS


In [18]:
!pip install PyMuPDF


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 36.6 MB/s eta 0:00:00


In [19]:
import sys, fitz

In [32]:
fname = '/content/drive/MyDrive/23-01-24_model/Resume (1).pdf'

In [33]:
doc = fitz.open(fname)
doc


Document('/content/drive/MyDrive/23-01-24_model/Resume (1).pdf')

In [34]:
text = " "
for page in doc:
  text = text + str(page.get_text())

In [35]:
text = ' '.join(text.split())

In [36]:
text

'Naman Grover +91 8700710350 / ngrover.241@gmail.com / Delhi, India / LinkedIn Knowledgeable in various avenues of Machine Learning. Ability to develop suitable Product Codes and Algorithms. Proactive with a problem-solving attitude. Ability to build innovative AI solutions for real-life concerns. WORK EXPERIENCE Asmadiya Technologies - AI/ML Engineer Jan 2021 – April 2021 / Pune, India • Researched, prototyped, built features and optimized the state-of-the-art deep learning techniques like sequence to sequence, LSTMs and attention using scikit-learn, keras and tensorflow for english-french machine translation. • Applied various transfer-learning techniques using pre-trained word-embeddings like Glove, Bert, Universal Sentence Encoder for text similarity tasks. • Developed text mining pipelines to analyze large unstructured datasets with Latent Dirichlet Allocation (LDA), text classification and sentiment analysis. • Collaborated with colleagues and executed other duties as assigned by

In [37]:
doc = nlp(text)

for ent in doc.ents:
  print(ent.text, "->>>>",ent.label_)

Naman Grover ->>>> NAME
SKILLS Programming Language: Python, C++; Tools and Frameworks: Pandas, Scikit-learn, Dash, Django, MySQL, Tensorflow, Tflite, Tensorboard, Huggingface; Data Science and Analytics: Numpy, Matplotlib, Seaborn, Pandas, Vaex, NLTK, ->>>> SKILLS
